# Load tests

> Performance testing for apswutils using different configuration

Plan: 

1. Create a single db load test that uses threading to perform a high volume of writes and some reads to replicate behavior under load by many users. Lock errors will be append to a simple text file called 'lock-errors.txt'
2. Extract that from the cell and feed that into timeit and/or cProfile in another cell

In [ ]:
import threading
import random
from string import ascii_letters
from apswutils.db import Database, Table
import timeit
import inspect

In [ ]:
def get_table(reset=False):
    db = Database("load.db")
    if reset:
        for t in db.tables: t.drop()
    users = Table(db, 'Users')
    users.create(columns=dict(id=int, name=str), transform=True, pk='id')
    return users

In [ ]:
users = get_table(reset=False)
users

<Table Users (id, name)>

In [ ]:
def get_random_name(length=10):
    return ''.join(random.choice(ascii_letters) for _ in range(length))
get_random_name()

'SCyFLvRxqC'

In [ ]:
def get_user(id): return users.get(id)
def set_user(id,name): return users.insert({'name':name},pk='id')

In [ ]:
def db_worker():
    record = set_user(random.randint(1,1000), get_random_name()).result[0]
    record['type'] = 'write'
    return record
db_worker()

{'id': 1, 'name': 'BqjlFTjvLU', 'type': 'write'}

In [ ]:
def db_worker_batch(size=100):
    for i in range(size): db_worker()

In [ ]:
def run_concurrent_workers(n_threads=10):
    print(users.count)
    threads = []
    for _ in range(n_threads):
        t = threading.Thread(target=db_worker_batch)
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()
    print(users.count)
    print('-------')

In [ ]:
run_concurrent_workers(1)

555201
555301
-------


In [ ]:
setup = """
import threading
import random
from string import ascii_letters
from apswutils.db import Database, Table
"""
setup += f'\n{inspect.getsource(get_table)}'
setup += f'\nusers = get_table()'
setup += f'\n{inspect.getsource(get_random_name)}'
setup += f'\n{inspect.getsource(get_user)}'
setup += f'\n{inspect.getsource(set_user)}'
setup += f'\n{inspect.getsource(db_worker)}'
setup += f'\n{inspect.getsource(db_worker_batch)}'
setup += f'\n{inspect.getsource(run_concurrent_workers)}'

# print(setup)

In [ ]:
test_code = """run_concurrent_workers(1000)"""

In [ ]:
print(timeit.repeat(stmt=test_code, setup=setup, number=1))

555301
655301
-------
655301
755301
-------
755301
855301
-------
855301
955301
-------
955301
1055301
-------
[291.8241256250185, 300.83988204202615, 285.32696783400024, 269.2562499170017, 268.16414191701915]
